In [75]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [270]:
import urllib.request

In [271]:
from datetime import datetime
import time

def get_epoch_timestamp(date_str):
    return int(time.mktime(datetime.strptime(date_str, '%Y-%m-%d').timetuple()))
# End of get_epoch_timestamp(..).

## Get Data from Facebook pages

In [204]:
import pandas as pd

app_id     = '145212799398035'                  
app_secret = '7efdb01e0e5e9400bd899b505815264f' 
page_id_list = ["SamsungMobile", "Nokiamobile", "sonymobile", "LGMobile", "HTC", "moto", "HuaweiMobileSG", "LenovoSmartphone", "xiaomiglobal", "opposingapore", "oneplussingapore", "meizuofficial", "BlackBerryMobile", "AlcatelmobileSingapore", "ZTECorp", "ToshibaUSA", "MyXOLO", "lavamobiles", "BLU.Products", "GioneeGlobal", "vivosingapore", "yuplaygod", "Maxwest.Telecom", "plummobile"]

In [201]:
from datetime import datetime
import json
import time
import urllib.request

def get_epoch_timestamp(date_str):
    return int(time.mktime(datetime.strptime(date_str, '%Y-%m-%d').timetuple()))
# End of get_epoch_timestamp(..).

def get_fb_page_json_data(page_id, 
                          access_token, 
                          no_of_status_posts=5, 
                          likes_limit=100, 
                          comments_limit=100, 
                          reactions_limit=100, 
                          start_date=None, end_date=None):
    base = "https://graph.facebook.com/v2.6"
    node = "/" + page_id + "/posts" 
    parameters = "/?fields=message,link,created_time,type,name,id,likes.limit(%s).summary(true),comments.limit(%s).summary(true),reactions.limit(%s).summary(true),shares&limit=%s&access_token=%s" % (likes_limit, comments_limit, reactions_limit, no_of_status_posts, access_token)
    if start_date is not None:
        parameters += '&since=%d' % get_epoch_timestamp(start_date)
    if end_date is not None:
        parameters += '&until=%d' % (get_epoch_timestamp(end_date) + 24 * 3600 - 1)
    url = base + node + parameters
    print('URL:\n%s' % url)
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    return response.read().decode('utf-8')

In [202]:
# Another way to piece 2 strings together (with a "pipe" character in the middle)
access_token = app_id + '|' +  app_secret
access_token

'145212799398035|7efdb01e0e5e9400bd899b505815264f'

### Convert JSON String to Dictionary Object

In [203]:
pd.set_option('display.max_colwidth', 150)
import json
import pandas as pd

for phone_brand in page_id_list:
    print("Doing " + phone_brand)
    posts = []
    json_data = get_fb_page_json_data(str(phone_brand), access_token, 
                              no_of_status_posts=100,
                              start_date='2016-01-01', end_date='2016-03-31')
    print("Got here")
    dict_data = json.loads(json_data)
    for post in dict_data['data']:
        dd = {}
        try:
            dd['message'] = post['message']
            dd['id'] = post['id']
            dd['type'] = post['type'] 
            dd['link'] =  post['link'] 
            dd['likes'] = post['likes']['summary']['total_count'] 
            dd['comments'] = post['comments']['summary']['total_count'] 
            dd['created_time'] =  post['created_time'] 
            dd['shares'] = post['shares']['count']
            posts.append(dd)
        except:
            continue
    # End of for loop.
    posts_df = pd.DataFrame(posts)
    posts_df.to_csv(phone_brand + '_2016Q1' + '.csv')

    


Doing Maxwest.Telecom
URL:
https://graph.facebook.com/v2.6/Maxwest.Telecom/posts/?fields=message,link,created_time,type,name,id,likes.limit(100).summary(true),comments.limit(100).summary(true),reactions.limit(100).summary(true),shares&limit=100&access_token=145212799398035|7efdb01e0e5e9400bd899b505815264f&since=1451577600&until=1459439999
Got here
Doing plummobile
URL:
https://graph.facebook.com/v2.6/plummobile/posts/?fields=message,link,created_time,type,name,id,likes.limit(100).summary(true),comments.limit(100).summary(true),reactions.limit(100).summary(true),shares&limit=100&access_token=145212799398035|7efdb01e0e5e9400bd899b505815264f&since=1451577600&until=1459439999
Got here
